In [1]:
import numpy as np
from numpy import array as arr

In [2]:
import sys
import socket
if socket.gethostname() == 'Birks-iMac.fritz.box':
    sys.path.append('/Users/Birk/Documents/Programmieren/Python/Eigener Code')
    sys.path.append('/Users/Birk/Documents/Programmieren/GitHub/experiment-code/pap')
else:
    sys.path.append('/Home/ab227/linux/Eigener Code')
from pap import _rundung  # Für _größenordnung Test

# Entwicklung interner Funktionen für pap.py

## Diverses

### _negativ_wird_null

In [3]:
'''
Benötigt von:
* pap.resultat()


Benutzt:
* numpy as np
'''


def _negativ_wird_null(array):
    '''
    "Rampenfunktion": Die Werte von array, die negativ sind, werden durch 0 ersetzt.
    
    
    Argument
    --------
    array : np.ndarray (number_like)
    '''
    
    
    nullen = np.full(np.shape(array), 0)
    return np.where(array > 0, array, nullen)

### _istbool 

In [4]:
'''
Benötigt von:
* pap.resultat()
'''


def _istbool(wert, bool_wert:bool):
    '''
    Überprüft ob wert wirklich bool_wert (True oder False) ist und nicht nur 1 oder 0.
    '''
    
    return (wert == bool_wert and isinstance(wert, bool))    

### _listen_transponieren 

In [5]:
'''
Benötigt von:
* pap.vergleichstabelle()
'''


def _listen_transponieren(listen_matrix):
    transponierte_matrix = []
    for i in range(len(listen_matrix[0])):
        zeile = []
        for j in range(len(listen_matrix)):
            zeile.append(listen_matrix[j][i])
        transponierte_matrix.append(zeile)
    return transponierte_matrix

In [6]:
# Test: _listen_transponieren()

test_listen_matrix = [['1', '2'], ['3', '4'], ['5', '6']]
test_listen_matrix = _listen_transponieren(test_listen_matrix)

In [7]:
test_werte = arr([[1.3, 0.2, 1.2, 0.3], [2.3, None, 2.5, 0.03], [20, 2.3, 17.3, None], [7.4e5, None, 7.6e5, None], 
                  [None, 123, 489, 45], [85, 2.3, None, 3.0], [None, 3.4, None, 3.2], [None, None, None, None]])
test_kein_wert = test_werte == None
none_stellen = np.where(test_kein_wert)
nullen = np.zeros(np.shape(test_werte))
test_werte = np.where(test_kein_wert, nullen, test_werte)

print(test_werte)
print(none_stellen)
ex_none = []
theo_none = []
for i in range(len(none_stellen[0])):
    if none_stellen[1][i] == 0:
        ex_none.append(none_stellen[0][i])
    if none_stellen[1][i] == 2:
        theo_none.append(none_stellen[0][i])
print(ex_none, theo_none)
'''
none_indices = arr(none_stellen).T
for i in none_indices:
    i = (i[0], i[1])
    test_werte[i] = 0.0
print(test_werte)
#print(test_werte[none_stellen] = 0)
''';

[[1.3 0.2 1.2 0.3]
 [2.3 0.0 2.5 0.03]
 [20 2.3 17.3 0.0]
 [740000.0 0.0 760000.0 0.0]
 [0.0 123 489 45]
 [85 2.3 0.0 3.0]
 [0.0 3.4 0.0 3.2]
 [0.0 0.0 0.0 0.0]]
(array([1, 2, 3, 3, 4, 5, 6, 6, 7, 7, 7, 7]), array([1, 3, 1, 3, 0, 2, 0, 2, 0, 1, 2, 3]))
[4, 6, 7] [5, 6, 7]


## Rundung u.Ä.

### _größenordnung

In [8]:
'''
Benötigt von:
* pap.vergleichstabelle()
* pap._nachkommastelle()
* pap._erste_ziffer()


Benötigt:
* numpy als np
* np.array() als arr()
'''


def _größenordnung(zahlen, art = int):
    if art == int:
        art = 'int64'
    elif art == float:
        art = 'float64'
    einsen = np.ones(np.shape(zahlen))   # 1 hat Größenordnung 0 (welches auch 0 hier haben soll).
    zahlen_kompatibel = np.where(zahlen == 0, einsen, zahlen)
    return arr(np.floor(np.log10(np.abs(zahlen_kompatibel))), dtype = art)

### _erste_ziffer 

In [9]:
'''
Benötigt von:
* pap._nachkommastelle()


Benötigt:
* numpy als np

* pap._größenorndung()
'''


def _erste_ziffer(zahlen, art = float):
    größenordnungen = _größenordnung(zahlen, art = float)
    ziffern = np.abs(zahlen / 10**größenordnungen)
    if art == float:
        return ziffern
    if art == int:
        return np.int_(ziffern)
    

In [10]:
# Test: _erste_ziffer()

print(_erste_ziffer(arr([1.293, 37, 0.9304, -1.60, -0.999999, 0.4, 0.6, -0.4, -0.6])))     
print(_erste_ziffer(arr([1.293, 37, 0.9304, -1.60, -0.999999, 0.4, 0.6, -0.4, -0.6]), art = int))

[1.293   3.7     9.304   1.6     9.99999 4.      6.      4.      6.     ]
[1 3 9 1 9 4 5 4 5]


### _nackommastelle

In [11]:
'''
Benötigt von:
* pap.vergleichstabelle()


Benötigt:
* numpy als np

* _größenordnung
* _erste_ziffer
'''


def _nachkommastelle(zahlen, sig_stellen = 1, sig_grenze = 1.0):
    
    über_der_grenze = _erste_ziffer(zahlen) >= sig_grenze
    sig_stellen_normal = np.full(np.shape(zahlen), sig_stellen)
    signifikante_stellen = np.where(über_der_grenze, sig_stellen_normal, sig_stellen_normal + 1)
    größenordnungen = _größenordnung(zahlen, art = float)
    
    nachkommastellen = -größenordnungen + signifikante_stellen - 1
    
    
    return nachkommastellen

In [12]:
# Test: _nachkommastelle(), _größenordnung()

zahlliste = arr([1e100, 3849184e38, -1.3984820e5, 48024.304, -40000, -39999.0, 1023, 999.9999, 483, -395.00, 
                 23.30489, 1.3848, 4.09, 3.688, -0.839, 0.094, 0.0003848, -0.00004848, 5.4839e-5, -3.9485e-7, 
                 7.38e-99])

nachkommastellen = _nachkommastelle(zahlliste, sig_grenze = 4.0)

print(_größenordnung(zahlliste))
print(nachkommastellen)
zahlliste_gerundet = _rundung(zahlliste, np.int_(nachkommastellen))
print(zahlliste_gerundet)

[100  44   5   4   4   4   3   2   2   2   1   0   0   0  -1  -2  -4  -5
  -5  -7 -99]
[-99. -43.  -4.  -4.  -4.  -3.  -2.  -2.  -2.  -1.   0.   1.   0.   1.
   1.   2.   5.   5.   5.   8.  99.]
[ 1.0e+100  3.8e+044 -1.4e+005  5.0e+004 -4.0e+004 -4.0e+004  1.0e+003
  1.0e+003  5.0e+002 -4.0e+002  2.3e+001  1.4e+000  4.0e+000  3.7e+000
 -8.0e-001  9.0e-002  3.8e-004 -5.0e-005  5.0e-005 -3.9e-007  7.0e-099]


## String-Formatierung 

### _füllen

In [13]:
'''
Benötigt von:
* pap.vergleichstabelle()
* pap._zahlen_ausrichtung()
'''


def _füllen(string, index = 0,  menge = 1, füllzeichen = ' '):
    if index == 'rechts':
        index = len(string)
    elif index == 'links':
        index = 0
    string_neu = string[:index] + füllzeichen * menge + string[index:]
    return string_neu

In [14]:
# Test: _füllen()

print(repr(_füllen('hi', index = 1, menge = 2)))
print(repr(_füllen('hi', index = len('hi'), menge = 2)))
print(repr(_füllen('hi', index = 'rechts', menge = 2)))

'h  i'
'hi  '
'hi  '


### _entfernen 

In [15]:
'''
Benötigt von:
* pap._zahlen_ausrichtung
'''

def _entfernen(string, index, länge = 1):
    
    return string[:index] + string[(index + länge):]

In [16]:
_entfernen('Hi !', 2)

'Hi!'

### _zentrieren_rechts

In [17]:
'''
Benötigt von:
* pap.vergleichstabelle()
* pap._zahlen_ausrichtung()

Benötigt:
* numpy als np
'''


def _zentrieren_rechts(string, länge_gewünscht, füllzeichen = ' '):
    länge_string = len(string)
    if länge_gewünscht <= länge_string:
        return string
    else:
        füll_länge        = länge_gewünscht - länge_string
        füll_länge_links  = int(np.ceil(füll_länge / 2))
        füll_länge_rechts = int(np.floor(füll_länge / 2))
        string_zentriert  = füllzeichen * füll_länge_links + string + füllzeichen * füll_länge_rechts
        return string_zentriert

### _zahlen_ausrichtung 

In [18]:
'''
Benötigt von:
* pap.vergleichstabelle()


Benötigt:
* numpy als np

* pap._füllen()
* pap._entferne()
* pap._zentrieren_rechts()
'''


def _zahlen_ausrichtung(zahlen_liste):
    
    länge_abs = len(zahlen_liste)
    test_zahl    = [any(x in string for x in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']) 
                    for string in zahlen_liste]
    nicht_zahlen = [(i, zahlen_liste.pop(i))  for i in reversed(range(länge_abs))  if not test_zahl[i]][::-1]
        # Pflückt alle nicht-Zahlen von hinten nach vorne aus der zahlen_liste um deren richtige Indices zu 
        # bewahren.
    
    if len(zahlen_liste) != 0:
        länge_abs = len(zahlen_liste)
        anzahl = 9   #% (minus, vor_komma, komma, nach_komma, e, e-, nach_e, text, ende)

        test_minus   = ['-' in string[0] for string in zahlen_liste]
        test_komma   = ['.' in string for string in zahlen_liste]
        test_e       = ['e' in string for string in zahlen_liste]
        test_e_minus = ['e-' in string for string in zahlen_liste]
        test_text    = [' ' in string for string in zahlen_liste]
        längen_strings = [len(string) for string in zahlen_liste]
        indices = np.int_(np.zeros((länge_abs, anzahl)))


        for i in range(länge_abs):
            indices[i][-1] = längen_strings[i]
            if test_text[i]:
                indices[i][7] = zahlen_liste[i].find(' ')
            else:
                indices[i][7] = indices[i][-1]
            if test_e[i]:
                indices[i][4] = zahlen_liste[i].find('e')
                if test_e_minus[i]:
                    indices[i][5] = zahlen_liste[i].find('e-') + 1
                    indices[i][6] = indices[i][5] + 1
                else: 
                    indices[i][5:7] = [indices[i][4] + 1] * 2
            else:
                indices[i][4:7] = [indices[i][7]] * 3
            if test_komma[i]:
                indices[i][2] = zahlen_liste[i].find('.')
                indices[i][3] = indices[i][2] + 1
            else:
                indices[i][2:4] = [indices[i][4]] * 2
            if test_minus[i]:
                indices[i][1] = 1
            else:
                indices[i][1] = 0

        längen = (indices.T[1:] - indices.T[:-1]).T
        längen_max = np.max(längen, axis = 0)
        längen_diff = längen_max[np.newaxis:] - längen

        längen_auswahl = [7, 6, 5, 4, 3, 2, 1, 0]
        indices_auswahl = [7, 5, 5, 4, 4, 2, 0, 0]   #% auf jeden Fall kommentieren.
        for i in range(länge_abs):
            for j in range(len(längen_auswahl)):
                zahlen_liste[i] = _füllen(zahlen_liste[i], menge = längen_diff[i][längen_auswahl[j]], 
                                          index = indices[i][indices_auswahl[j]])
    
        test_leerzeichen = [' ' in string[0] for string in zahlen_liste]
        test_e_leer      = ['e ' in string if 'e' in string else True for string in zahlen_liste]
        if all(test_leerzeichen):
            for i in range(länge_abs):
                zahlen_liste[i] = zahlen_liste[i][1:]
        if all(test_e_leer) and any(test_e):
            index_e_string = test_e.index(True)
            index_nach_e   = zahlen_liste[index_e_string].index('e') + 1
            zahlen_liste = [_entfernen(string, index_nach_e) for string in zahlen_liste]
        
        länge_gesamt = len(zahlen_liste[0])
    else:
        länge_gesamt = max([len(string) for i, string in nicht_zahlen])
    for tupel in nicht_zahlen:
        zahlen_liste.insert(tupel[0], _zentrieren_rechts(tupel[1], länge_gesamt))
    
    return zahlen_liste

In [19]:
liste = ['1.003', '789e-3', '-', '', ' ', '-29.4834e100', '-0.000334', 'x', '53449', '-- %', '27 %', '-5.81 %', '1.239e-24 %']
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

'    1.003         '
'  789       e -3  '
'         -        '
'                  '
'                  '
'  -29.4834  e100  '
'   -0.000334      '
'         x        '
'53449             '
'       -- %       '
'   27            %'
'   -5.81         %'
'    1.239   e-24 %'


In [20]:
liste = ['-', '', ' ', 'x', '-- %']
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

'  - '
'    '
'    '
'  x '
'-- %'


In [21]:
liste = ['-']
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

'-'


In [22]:
liste = ['1.003', '789e-3', '-29.4834e100', '-0.000334', '53449', '27 %', '-5.81 %', '1.239e-24 %']
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

'    1.003         '
'  789       e -3  '
'  -29.4834  e100  '
'   -0.000334      '
'53449             '
'   27            %'
'   -5.81         %'
'    1.239   e-24 %'


In [23]:
liste = ['-29.4834e100']
liste_ausgerichtet = _zahlen_ausrichtung(liste)
for string in liste_ausgerichtet:
    print(repr(string))

'-29.4834e100'


In [24]:
liste = ['1', 'a', '2.0',  'b', '-32.9', 'c', '94e3', 'd', '2.87e-83', 'e']
print(liste)
print('')
test_zahl    = [any(x in string for x in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']) 
                    for string in liste]
test = []
for i in reversed(range(len(liste))):
    if (not test_zahl[i]):
        test.append((i, liste.pop(i)))
test.reverse()

print(test)
print(liste)
print('')

for i in range(len(test)):
    liste.insert(test[i][0], test[i][1])
        
print(liste)

['1', 'a', '2.0', 'b', '-32.9', 'c', '94e3', 'd', '2.87e-83', 'e']

[(1, 'a'), (3, 'b'), (5, 'c'), (7, 'd'), (9, 'e')]
['1', '2.0', '-32.9', '94e3', '2.87e-83']

['1', 'a', '2.0', 'b', '-32.9', 'c', '94e3', 'd', '2.87e-83', 'e']


In [25]:
liste = ['1', 'a', '2.0',  'b', '-32.9', 'c', '94e3', 'd', '2.87e-83' 'e']
test_zahl    = [any(x in string for x in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']) 
                    for string in liste]
#liste_neu = liste[test_zahl]
liste_neu = [(i, liste.pop(i)) for i in range(len(liste))[::-1] if not test_zahl[i]][::-1]
liste_neu.reverse()
print(liste_neu)

[(7, 'd'), (5, 'c'), (3, 'b'), (1, 'a')]


### _plus_minus

In [26]:
'''
Benötigt von:
* pap.vergleichstabelle()
'''


def _plus_minus(string_liste):
    plus_minus = []
    for i in range(len(string_liste)):
        if string_liste[i] == '':
            plus_minus.append('')
        elif all([zeichen == ' ' for zeichen in string_liste[i]]):
            plus_minus.append('   ')
        else:
            plus_minus.append(' ± ')
    return plus_minus

In [27]:
liste = [0, 1, True, 2.0, -3.4, 34e-90, '2', complex(0, 0), complex(1, 0), complex(0, 1), arr(3), arr(0.4), arr([2])[0], arr([3, -4])]
for element in liste:
    if type(element) == np.ndarray: 
        test_zahl = element.dtype in (np.int64, np.float64)
    else:
        test_zahl = isinstance(element, (int, float))
    print(element, test_zahl, type(element))

0 True <class 'int'>
1 True <class 'int'>
True True <class 'bool'>
2.0 True <class 'float'>
-3.4 True <class 'float'>
3.4e-89 True <class 'float'>
2 False <class 'str'>
0j False <class 'complex'>
(1+0j) False <class 'complex'>
1j False <class 'complex'>
3 True <class 'numpy.ndarray'>
0.4 True <class 'numpy.ndarray'>
2 False <class 'numpy.int64'>
[ 3 -4] True <class 'numpy.ndarray'>
